# softmax回归的简洁实现
这一节通过深度学习框架的高级API简便实现softmax回归模型。

In [3]:
import torch
from torch import nn
from d2l import torch as d2l
import warnings
warnings.filterwarnings('ignore')

# 构建数据集

In [2]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

# 初始化模型参数
softmax回归的输出层是一个全连接层，在输入的时候需要将特征矩阵展平：

In [4]:
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

# 重新审视softmax的实现
softmax函数$\hat y_j = \frac{\exp(o_j)}{\sum_k \exp(o_k)}$，
其中$\hat y_j$是预测的概率分布。
$o_j$是未规范化的预测$\mathbf{o}$的第$j$个元素。
如果$o_k$中的一些数值非常大，
那么$\exp(o_k)$可能大于数据类型容许的最大数字，即*上溢*（overflow）。
这将使分母或分子变为`inf`（无穷大），
最后得到的是0、`inf`或`nan`（不是数字）的$\hat y_j$。
在这些情况下，我们无法得到一个明确定义的交叉熵值。

尽管我们要计算指数函数，但我们最终在计算交叉熵损失时会取它们的对数。
通过将softmax和交叉熵结合在一起，可以避免反向传播过程中可能会困扰我们的数值稳定性问题。
如下面的等式所示，我们避免计算$\exp(o_j - \max(o_k))$，
而可以直接使用$o_j - \max(o_k)$，因为$\log(\exp(\cdot))$被抵消了。

$$
\begin{aligned}
\log{(\hat y_j)} & = \log\left( \frac{\exp(o_j - \max(o_k))}{\sum_k \exp(o_k - \max(o_k))}\right) \\
& = \log{(\exp(o_j - \max(o_k)))}-\log{\left( \sum_k \exp(o_k - \max(o_k)) \right)} \\
& = o_j - \max(o_k) -\log{\left( \sum_k \exp(o_k - \max(o_k)) \right)}.
\end{aligned}
$$

我们也希望保留传统的softmax函数，以备我们需要评估通过模型输出的概率。
但是，我们没有将softmax概率传递到损失函数中，
而是[**在交叉熵损失函数中传递未规范化的预测，并同时计算softmax及其对数**]，
这是一种类似["LogSumExp技巧"](https://en.wikipedia.org/wiki/LogSumExp)的聪明方式。

In [5]:
loss =nn.CrossEntropyLoss(reduction='none')

# 优化算法

In [6]:
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

# 训练

In [ ]:
num_epochs = 10
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

# 小结

* 使用深度学习框架的高级API，我们可以更简洁地实现softmax回归。
* 从计算的角度来看，实现softmax回归比较复杂。在许多情况下，深度学习框架在这些著名的技巧之外采取了额外的预防措施，来确保数值的稳定性。这使我们避免了在实践中从零开始编写模型时可能遇到的陷阱。

In [12]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils import data

In [10]:
trans = transforms.ToTensor()
data_train = torchvision.datasets.FashionMNIST(root='../data', train=True, transform=trans, download=True)
data_test = torchvision.datasets.FashionMNIST(root='../data', train=False, transform=trans, download=True)

In [11]:
loader_train = data.DataLoader(data_train, shuffle=True, batch_size=256)
loader_test = data.DataLoader(data_test, shuffle=True, batch_size=256)

In [15]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.flatten = nn.Flatten()
        self.Linear = nn.Linear(784, 10)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.Linear(x)
        return x
model = net()

In [16]:
loss = nn.CrossEntropyLoss()

In [17]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

In [22]:
num_epochs = 3
for epoch in range(num_epochs):
    loss_total = 0
    samples = 0
    for X, y in loader_train:     
        y_hat = model(X)
        loss_batch = loss(y_hat, y)
        optimizer.zero_grad()
        loss_batch.mean().backward()
        optimizer.step()
        loss_total += loss_batch
        samples += len(y)
    print(loss_total / samples)

tensor(0.0017, grad_fn=<DivBackward0>)
tensor(0.0017, grad_fn=<DivBackward0>)
tensor(0.0017, grad_fn=<DivBackward0>)
